# Sioux Falls example

## File paths

In [2]:
fldr = 'D:/release/Sample models/sioux_falls_2020_02_15'
proj_name = 'SiouxFalls.sqlite'

# remove the comments for the lines below to run the Chicago model example instead
# fldr = 'D:/release/Sample models/Chicago_2020_02_15'
# proj_name = 'chicagomodel.sqlite'

dt_fldr = '0_tntp_data'
prj_fldr = '1_project'
skm_fldr = '2_skim_results'
assg_fldr = '4_assignment_results'
dstr_fldr = '5_distribution_results'
frcst_fldr = '6_forecast'
ftr_fldr = '7_future_year_assignment'

## Some logging

In [3]:
import aequilibrae
from os.path import join
from aequilibrae import Parameters

p = Parameters()
p.parameters['system']['logging_directory'] =  fldr
p.write_back()

In [4]:
# To make sure the logging will go where it should, we reload aequilibrae
from importlib import reload
reload(aequilibrae)

<module 'aequilibrae' from 'D:/src/aequilibrae\\aequilibrae\\__init__.py'>

## Opening the project

In [5]:
# Imports
from aequilibrae.project import Project

In [6]:
project = Project()
project.load(join(fldr, prj_fldr, proj_name))

## Path computation

In [7]:
# imports
from aequilibrae.paths import PathResults, path_computation

In [8]:
# we build all graphs
project.network.build_graphs()
# We get warnings that several fields in the project are filled with NaNs.  Which is true, but we won't use those fields

D:/src/aequilibrae\aequilibrae\project\network\network.py:239: UserWarning: Fields were removed form Graph for being non-numeric: modes,link_type
  warn(f'Fields were removed form Graph for being non-numeric: {",".join(removed_fields)}')
D:/src/aequilibrae\aequilibrae\paths\graph.py:331: UserWarning: Field name has at least one NaN value.  Your computation may be compromised
  warn(f'Field {i} has at least one NaN value.  Your computation may be compromised')
D:/src/aequilibrae\aequilibrae\paths\graph.py:331: UserWarning: Field lanes has at least one NaN value.  Your computation may be compromised
  warn(f'Field {i} has at least one NaN value.  Your computation may be compromised')


In [9]:
# we grab the graph for cars
graph = project.network.graphs['c']

# let's say we want to minimize distance
graph.set_graph('distance')

# And will skim time and distance while we are at it
graph.set_skimming(['free_flow_time', 'distance'])

# And we will allow paths to be compute going through other centroids/centroid connectors
# required for the Sioux Falls network, as all nodes are centroids
graph.set_blocked_centroid_flows(False)

# instantiate a path results object and prepare it to work with the graph
res = PathResults()
res.prepare(graph)

# compute a path from node 2 to 13
path_computation(2, 13, graph, res)

In [10]:
# We can get the sequence of nodes we traverse
res.path_nodes

array([    2, 10294,  7860, 10295,  2594, 12722, 10297, 12732, 10299,
       10301, 12731, 10300, 10302, 10305,    13], dtype=int64)

In [11]:
# We can get the link sequence we traverse
res.path

array([    2, 28174, 20980, 28176,  4194, 38149, 28188, 38182, 28197,
       28206, 38178, 28200, 28209, 28219], dtype=int64)

In [12]:
# We can get the mileposts for our sequence of nodes
res.milepost

array([0.  , 0.5 , 0.74, 1.32, 1.73, 1.98, 2.49, 3.11, 3.17, 3.89, 4.17,
       4.92, 5.74, 6.86, 7.3 ])

In [13]:
# And We can the skims for our tree
res.skims

array([[ 2.856,  2.14 ,  0.   ],
       [ 0.   ,  0.   ,  0.   ],
       [ 1.736,  1.71 ,  0.   ],
       ...,
       [41.358, 21.44 ,  0.   ],
       [42.398, 21.96 ,  0.   ],
       [ 0.   ,  0.   ,  0.   ]])

In [14]:
# If we want to compute the path for a different destination and same origin, we can just do this
# It is way faster when you have large networks
res.update_trace(4)

In [15]:
res.path_nodes

array([    2, 10294,  7856,  2593, 10296,     4], dtype=int64)

## Skimming

In [16]:
from aequilibrae.matrix import AequilibraeData, AequilibraeMatrix
from aequilibrae.paths import NetworkSkimming, SkimResults

In [17]:
# from before
project = Project()
project.load(join(fldr, prj_fldr, proj_name))
project.network.build_graphs()

graph = project.network.graphs['c'] # we grab the graph for cars
graph.set_graph('free_flow_time') # let's say we want to minimize time
graph.set_skimming(['free_flow_time', 'distance']) # And will skim time and distance
graph = project.network.graphs['c'] # we grab the graph for cars
graph.set_blocked_centroid_flows(False)

In [18]:
# setup the object result
res = SkimResults()
res.prepare(graph)

In [19]:
# And run the skimming
skm = NetworkSkimming(graph, res)
skm.execute()

In [20]:
# The result is an AequilibraEMatrix object
skims = res.skims

# We can export to AEM and OMX
skims.export(join(fldr, skm_fldr, 'skimming_on_time.aem'))
skims.export(join(fldr, skm_fldr, 'skimming_on_time.omx'))

# Traffic assignment with skimming

In [21]:
from aequilibrae.matrix import AequilibraeMatrix
from aequilibrae.paths import TrafficAssignment, TrafficClass
from aequilibrae import logger
import logging

In [22]:
# from before
project = Project()
project.load(join(fldr, prj_fldr, proj_name))
project.network.build_graphs()

graph = project.network.graphs['c'] # we grab the graph for cars
graph.set_graph('free_flow_time') # let's say we want to minimize time
graph.set_skimming(['free_flow_time', 'distance']) # And will skim time and distance
graph.set_blocked_centroid_flows(False)

In [23]:
# Because assignment takes a long time, we want the log to be shown here
stdout_handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter("%(asctime)s;%(name)s;%(levelname)s ; %(message)s")
stdout_handler.setFormatter(formatter)
logger.addHandler(stdout_handler)

In [24]:
demand = AequilibraeMatrix()
demand.load(join(fldr, dt_fldr, 'demand.omx'))
demand.computational_view(['matrix']) # We will only assign one user class stored as 'matrix' inside the OMX file

assig = TrafficAssignment()

# Creates the assignment class
assigclass = TrafficClass(graph, demand)


# The first thing to do is to add at list of traffic classes to be assigned
assig.set_classes([assigclass])

assig.set_vdf("BPR")  # This is not case-sensitive # Then we set the volume delay function

assig.set_vdf_parameters({"alpha": "b", "beta": "power"}) # And its parameters

assig.set_capacity_field("capacity") # The capacity and free flow travel times as they exist in the graph
assig.set_time_field("free_flow_time")

# And the algorithm we want to use to assign
assig.set_algorithm('bfw')

# since I haven't checked the parameters file, let's make sure convergence criteria is good
assig.max_iter = 1000
assig.rgap_target = 0.00001

assig.execute() # we then execute the assignment

2020-03-01 21:15:59,002;aequilibrae;INFO ; bfw Assignment STATS
2020-03-01 21:15:59,002;aequilibrae;INFO ; Iteration, RelativeGap, stepsize
2020-03-01 21:16:01,761;aequilibrae;INFO ; 1,inf,1.0
2020-03-01 21:16:04,547;aequilibrae;INFO ; 2,0.8493316404205302,0.22717528747744073
2020-03-01 21:16:07,322;aequilibrae;INFO ; 3,0.6477704743509676,0.20575249566656492
2020-03-01 21:16:10,119;aequilibrae;INFO ; 4,0.4217031292016379,0.16174827631347571
2020-03-01 21:16:12,952;aequilibrae;INFO ; 5,0.26227767252701367,0.17031714733107065
2020-03-01 21:16:15,836;aequilibrae;INFO ; 6,0.1577172341234456,0.19813230198582563
2020-03-01 21:16:18,914;aequilibrae;INFO ; 7,0.10597604163296806,0.22001884786354575
2020-03-01 21:16:21,724;aequilibrae;INFO ; 8,0.07806471093424068,0.2204973521004745
2020-03-01 21:16:24,738;aequilibrae;INFO ; 9,0.058529977246835646,0.3011289345907604
2020-03-01 21:16:27,736;aequilibrae;INFO ; 10,0.05337559671278759,0.42517114327595057
2020-03-01 21:16:30,877;aequilibrae;INFO ; 11,

2020-03-01 21:20:30,384;aequilibrae;INFO ; 95,0.0005484255246145709,0.06604715551590179
2020-03-01 21:20:33,187;aequilibrae;INFO ; 96,0.00047524860335592295,0.07385950737039605
2020-03-01 21:20:36,114;aequilibrae;INFO ; 97,0.0005396452803955574,0.09543881844338928
2020-03-01 21:20:38,788;aequilibrae;INFO ; 98,0.0005054966943127236,0.08922441419062521
2020-03-01 21:20:41,454;aequilibrae;INFO ; 99,0.0004543908272645558,0.057571853669656184
2020-03-01 21:20:44,302;aequilibrae;INFO ; 100,0.0004394444785869702,0.0755187646484357
2020-03-01 21:20:47,034;aequilibrae;INFO ; 101,0.0004893584371345484,0.04309486619998589
2020-03-01 21:20:49,775;aequilibrae;INFO ; 102,0.0004510213171089349,0.08690490817584984
2020-03-01 21:20:52,859;aequilibrae;INFO ; 103,0.0005094751437353294,0.09250364687363398
2020-03-01 21:20:55,936;aequilibrae;INFO ; 104,0.00043756142931314807,0.08848771086213184
2020-03-01 21:20:58,969;aequilibrae;INFO ; 105,0.00044410556574803493,0.06246648703469412
2020-03-01 21:21:01,862

2020-03-01 21:25:15,935;aequilibrae;INFO ; 187,9.819803104661292e-05,0.01068394740780448
2020-03-01 21:25:19,606;aequilibrae;INFO ; 188,0.00011259599866970329,0.01271698045569262
2020-03-01 21:25:23,719;aequilibrae;INFO ; 189,0.0001181015430035482,0.01181246666219826
2020-03-01 21:25:27,328;aequilibrae;INFO ; 190,0.00010779704659305938,0.015260182787936673
2020-03-01 21:25:30,308;aequilibrae;INFO ; 191,0.0001059096925942023,0.01274123711273735
2020-03-01 21:25:33,819;aequilibrae;INFO ; 192,0.00010676403435666863,0.010737841426614264
2020-03-01 21:25:37,163;aequilibrae;INFO ; 193,0.00010287520427119918,0.012860754613563768
2020-03-01 21:25:40,056;aequilibrae;INFO ; 194,0.00011073556091257418,0.013162819036844612
2020-03-01 21:25:43,190;aequilibrae;INFO ; 195,0.00010394478399557855,0.012879212147447777
2020-03-01 21:25:46,058;aequilibrae;INFO ; 196,9.8652000078642e-05,0.011651532481822212
2020-03-01 21:25:48,969;aequilibrae;INFO ; 197,9.228925148578689e-05,0.007852303132301893
2020-03-01

2020-03-01 21:29:59,314;aequilibrae;INFO ; 279,5.813424283847301e-05,0.006518445500184196
2020-03-01 21:30:02,166;aequilibrae;INFO ; 280,5.6225578932174604e-05,0.006676296332407237
2020-03-01 21:30:04,938;aequilibrae;INFO ; 281,5.47293809447678e-05,0.005216788024252037
2020-03-01 21:30:07,817;aequilibrae;INFO ; 282,4.866737380318216e-05,0.0047789789739561515
2020-03-01 21:30:10,851;aequilibrae;INFO ; 283,4.859241463638052e-05,0.005556291654116051
2020-03-01 21:30:13,777;aequilibrae;INFO ; 284,4.07993729189582e-05,0.003054291440786463
2020-03-01 21:30:16,494;aequilibrae;INFO ; 285,4.586046420859943e-05,0.005319430416137654
2020-03-01 21:30:19,217;aequilibrae;INFO ; 286,4.361533466698868e-05,0.00365146339751304
2020-03-01 21:30:21,973;aequilibrae;INFO ; 287,4.506458983576297e-05,0.006513180516731543
2020-03-01 21:30:24,674;aequilibrae;INFO ; 288,5.2906736994630056e-05,0.006360793592645648
2020-03-01 21:30:27,400;aequilibrae;INFO ; 289,4.7336068595981104e-05,0.008980716439126953
2020-03-0

2020-03-01 21:34:20,462;aequilibrae;INFO ; 370,3.779942747831624e-05,0.0033198080785328886
2020-03-01 21:34:23,433;aequilibrae;INFO ; 371,3.507047628057561e-05,0.00343001401710046
2020-03-01 21:34:26,288;aequilibrae;INFO ; 372,3.147253674181106e-05,0.003862331722458745
2020-03-01 21:34:28,998;aequilibrae;INFO ; 373,2.926490223478143e-05,0.0036742605244068777
2020-03-01 21:34:31,746;aequilibrae;INFO ; 374,3.205190954297514e-05,0.0034712737923868514
2020-03-01 21:34:34,585;aequilibrae;INFO ; 375,2.7923876034759823e-05,0.003271986304704208
2020-03-01 21:34:37,383;aequilibrae;INFO ; 376,2.6947967659143814e-05,0.003583477451425545
2020-03-01 21:34:40,301;aequilibrae;INFO ; 377,3.146808836617256e-05,0.003906898553199818
2020-03-01 21:34:43,254;aequilibrae;INFO ; 378,3.156665952446095e-05,0.0049114472685019344
2020-03-01 21:34:46,017;aequilibrae;INFO ; 379,3.0336388871655833e-05,0.004644508143722447
2020-03-01 21:34:48,751;aequilibrae;INFO ; 380,3.298617078487321e-05,0.005849394278669085
2020

2020-03-01 21:38:38,935;aequilibrae;INFO ; 461,2.4241800062198384e-05,0.002975935065224544
2020-03-01 21:38:41,637;aequilibrae;INFO ; 462,2.3799975781369544e-05,0.0025189395753495362
2020-03-01 21:38:44,330;aequilibrae;INFO ; 463,2.406480293149752e-05,0.0026206762035203357
2020-03-01 21:38:47,025;aequilibrae;INFO ; 464,2.038995128332381e-05,0.002343473060567432
2020-03-01 21:38:49,721;aequilibrae;INFO ; 465,2.5094861625573868e-05,0.0025553160385076074
2020-03-01 21:38:52,562;aequilibrae;INFO ; 466,1.9826191100089393e-05,0.0022924763585429513
2020-03-01 21:38:55,437;aequilibrae;INFO ; 467,2.1861702585866665e-05,0.001965020375917525
2020-03-01 21:38:58,158;aequilibrae;INFO ; 468,1.9899135280387922e-05,0.0024191932189823466
2020-03-01 21:39:00,844;aequilibrae;INFO ; 469,1.9371288393683762e-05,0.0021642464781269004
2020-03-01 21:39:03,545;aequilibrae;INFO ; 470,2.0680084738771913e-05,0.0020641746305487905
2020-03-01 21:39:06,340;aequilibrae;INFO ; 471,2.1002147999842436e-05,0.0022021920959

2020-03-01 21:42:56,298;aequilibrae;INFO ; 551,1.922642453371732e-05,0.002358540923836584
2020-03-01 21:42:59,527;aequilibrae;INFO ; 552,1.7351991803281362e-05,0.003010410532704903
2020-03-01 21:43:02,841;aequilibrae;INFO ; 553,1.740657993784294e-05,0.00236607734196373
2020-03-01 21:43:06,072;aequilibrae;INFO ; 554,1.680587795761432e-05,0.0018538732879666896
2020-03-01 21:43:09,289;aequilibrae;INFO ; 555,1.731400463253108e-05,0.0020065614375377335
2020-03-01 21:43:12,378;aequilibrae;INFO ; 556,1.6876293426628097e-05,0.002016244590120083
2020-03-01 21:43:15,457;aequilibrae;INFO ; 557,1.4595375253592138e-05,0.001525736535537764
2020-03-01 21:43:18,416;aequilibrae;INFO ; 558,1.5203382551693248e-05,0.0015796391329184003
2020-03-01 21:43:21,539;aequilibrae;INFO ; 559,1.5706749909290584e-05,0.0017471267555994009
2020-03-01 21:43:24,853;aequilibrae;INFO ; 560,1.562221377048636e-05,0.001584822629583108
2020-03-01 21:43:27,852;aequilibrae;INFO ; 561,1.624670703621111e-05,0.0019444241476864817
2

2020-03-01 21:48:06,249;aequilibrae;INFO ; 641,1.4111707467713706e-05,0.0012964942658903937
2020-03-01 21:48:09,820;aequilibrae;INFO ; 642,1.335126486355198e-05,0.0013598692981380098
2020-03-01 21:48:13,446;aequilibrae;INFO ; 643,1.266179881616389e-05,0.0014697456453934795
2020-03-01 21:48:17,316;aequilibrae;INFO ; 644,1.3303045500397876e-05,0.001853842440967689
2020-03-01 21:48:21,803;aequilibrae;INFO ; 645,1.3670216671487814e-05,0.0018218672771857871
2020-03-01 21:48:25,872;aequilibrae;INFO ; 646,1.415623862056236e-05,0.0017854761629306434
2020-03-01 21:48:29,466;aequilibrae;INFO ; 647,1.2974214602996775e-05,0.0020114513980437117
2020-03-01 21:48:32,990;aequilibrae;INFO ; 648,1.3760843786878268e-05,0.0016230141074026108
2020-03-01 21:48:36,682;aequilibrae;INFO ; 649,1.3184423142873145e-05,0.0013379333763471571
2020-03-01 21:48:40,255;aequilibrae;INFO ; 650,1.3888393762215245e-05,0.001946383954830659
2020-03-01 21:48:43,873;aequilibrae;INFO ; 651,1.2469561763288599e-05,0.0015088840615

### Save outputs

In [25]:
# The link flows are easy to export.
# we do so for csv and AequilibraEData
assigclass.results.save_to_disk(join(fldr, assg_fldr, 'link_flows_c.csv'), output="loads")
assigclass.results.save_to_disk(join(fldr, assg_fldr, 'link_flows_c.aed'), output="loads")

In [26]:
# the skims are easy to get.

# The blended one are here
avg_skims = assigclass.results.skims

# The ones for the last iteration are here
last_skims = assigclass._aon_results.skims

# Assembling a single final skim file can be done like this
# We will want only the time for the last iteration and the distance averaged out for all iterations
kwargs = {'file_name': join(fldr,assg_fldr, 'skims.aem'),
          'zones': graph.num_zones,
          'matrix_names': ['time_final', 'distance_blended']}

# Create the matrix file
out_skims = AequilibraeMatrix()
out_skims.create_empty(**kwargs)
out_skims.index[:] = avg_skims.index[:]

# Transfer the data
 # The names of the skims are the name of the fields
out_skims.matrix['time_final'][:,:] = last_skims.matrix['free_flow_time'][:,:]
# It is CRITICAL to assign the matrix values using the [:,:]
out_skims.matrix['distance_blended'][:,:] = avg_skims.matrix['distance'][:,:]

out_skims.matrices.flush() # Make sure that all data went to the disk

# Export to OMX as well
out_skims.export(join(fldr,assg_fldr, 'skims.omx'))

    

# Trip distribution

### Calibration

We will calibrate synthetic gravity models using the skims for TIME that we just generated

In [27]:
import numpy as np
from aequilibrae.distribution import GravityCalibration
from aequilibrae.matrix import AequilibraeMatrix

In [28]:
# We need the demand
demand = AequilibraeMatrix()
demand.load(join(fldr, dt_fldr, 'demand.omx'))

# And the skims
imped = AequilibraeMatrix()
imped.load(join(fldr,assg_fldr, 'skims.aem'))

In [29]:
# But before using the data, let's get some impedance for the intrazonals
# Let's assume it is 75% of the closest zone

# If we run the code below more than once, we will be overwriting the diagonal values with non-sensical data
# so let's zero it first
np.fill_diagonal(imped.matrix['time_final'], 0)

# We compute it with a little bit of NumPy magic
intrazonals = np.amin(imped.matrix['time_final'], where=imped.matrix['time_final']>0, initial=imped.matrix['time_final'].max(), axis=1)
intrazonals *= 0.75

# Then we fill in the impedance matrix
np.fill_diagonal(imped.matrix['time_final'], intrazonals)


In [30]:
# We set the matrices forbeing used in computation
imped.computational_view(['time_final'])
demand.computational_view(['matrix'])

In [31]:
from math import log10, floor
def plot_tlfd(demand, skim, name):
    import matplotlib.pyplot as plt
    b = floor(log10(skim.shape[0]) * 10)
    n, bins, patches = plt.hist(np.nan_to_num(skim.flatten(),0), bins = b, weights=np.nan_to_num(demand.flatten()), density=False, facecolor='g', alpha=0.75)

    plt.xlabel('Trip length')
    plt.ylabel('Probability')
    plt.title('Trip-length frequency distribution')
    plt.savefig(name, format="png")
    plt.clf()

In [32]:
for function in ['power', 'expo']:
    model = GravityCalibration(matrix=demand, impedance=imped, function=function, nan_as_zero=True)
    model.calibrate()
    
    # we save the model
    model.model.save(join(fldr, dstr_fldr, f'{function}_model.mod'))
    
    # We save a trip length frequency distribution image
    plot_tlfd(model.result_matrix.matrix_view, imped.matrix_view,join(fldr, dstr_fldr, f'{function}_tfld.png') )
    
    # We can save the result of applying the model as well
    # we can also save the calibration report
    with open(join(fldr, dstr_fldr, f'{function}_convergence.log'), 'w') as otp:
        for r in  model.report:
            otp.write(r+'\n')

D:/src/aequilibrae\aequilibrae\distribution\ipf.py:193: RuntimeWarning: invalid value encountered in true_divide
  f = np.divide(targets, marginals)  # We compute the factors
D:/src/aequilibrae\aequilibrae\distribution\ipf.py:193: RuntimeWarning: invalid value encountered in true_divide
  f = np.divide(targets, marginals)  # We compute the factors


<Figure size 432x288 with 0 Axes>

In [33]:
# We save a trip length frequency distribution image
plot_tlfd(demand.matrix_view, imped.matrix_view,join(fldr, dstr_fldr, 'demand_tfld.png') )

<Figure size 432x288 with 0 Axes>

# Forecast

* We create a set of *'future'* vectors using some random growth factors
* We apply the model for inverse power, as the TFLD seems to be a better fit for the actual one

In [34]:
from aequilibrae.distribution import Ipf, GravityApplication, SyntheticGravityModel, Ipf
from aequilibrae.matrix import AequilibraeData, AequilibraeMatrix
import numpy as np

In [35]:
# We compute the vectors from our matrix
mat = AequilibraeMatrix()

mat.load(join(fldr, dt_fldr, 'demand.omx'))
mat.computational_view()
origins = np.sum(mat.matrix_view, axis=1)
destinations = np.sum(mat.matrix_view, axis=0)

args = {'file_path':join(fldr,  frcst_fldr, 'synthetic_future_vector.aed'),
        "entries": mat.zones, 
        "field_names": ["origins", "destinations"],
    "data_types": [np.float64, np.float64], 
        "memory_mode": False}

vectors = AequilibraeData()
vectors.create_empty(**args)

vectors.index[:] =mat.index[:]

# Then grow them with some random growth between 0 and 10% - Plus balance them
vectors.origins[:] = origins * (1+ np.random.rand(vectors.entries)/10)
vectors.destinations[:] = destinations * (1+ np.random.rand(vectors.entries)/10)
vectors.destinations *= vectors.origins.sum()/vectors.destinations.sum()

In [36]:
# Impedance 
imped = AequilibraeMatrix()
imped.load(join(fldr,assg_fldr, 'skims.aem'))
imped.computational_view(['time_final'])

# We want the main diagonal to be zero
np.fill_diagonal(imped.matrix_view, np.nan)

In [37]:
for function in ['power', 'expo']:
    model = SyntheticGravityModel()
    model.load(join(fldr, dstr_fldr, f'{function}_model.mod'))

    outmatrix = join(fldr,frcst_fldr, f'demand_{function}_model.aem') 
    apply = GravityApplication()
    args = {"impedance": imped,
            "rows": vectors,
            "row_field": "origins",
            "model": model,
            "columns": vectors,
            "column_field": "destinations",
            "output": outmatrix,
            "nan_as_zero":True
            }

    gravity = GravityApplication(**args)
    gravity.apply()

    #We get the output matrix and save it to OMX too
    resm = AequilibraeMatrix()
    resm.load(outmatrix)
    resm.export(join(fldr,frcst_fldr, f'demand_{function}_model.omx'))

### We now run IPF for the future vectors

In [38]:
demand = AequilibraeMatrix()
demand.load(join(fldr, dt_fldr, 'demand.omx'))
demand.computational_view()

args = {'matrix': demand,
        'rows': vectors,
        'columns': vectors,
        'column_field': "destinations",
        'row_field': "origins",
        'nan_as_zero': True}

ipf = Ipf(**args)
ipf.fit()

output = AequilibraeMatrix()
output.load(ipf.output.file_path)

output.export(join(fldr,frcst_fldr, 'demand_ipf.aem'))
output.export(join(fldr,frcst_fldr, 'demand_ipf.omx'))


# Future traffic assignment

In [39]:
from aequilibrae.matrix import AequilibraeMatrix
from aequilibrae.paths import TrafficAssignment, TrafficClass
from aequilibrae import logger
import logging

In [40]:
logger.info('\n\n\n TRAFFIC ASSIGNMENT FOR FUTURE YEAR')

2020-03-01 21:51:43,279;aequilibrae;INFO ; 


 TRAFFIC ASSIGNMENT FOR FUTURE YEAR


In [41]:
# from before
project = Project()
project.load(join(fldr, prj_fldr, proj_name))
project.network.build_graphs()

graph = project.network.graphs['c'] # we grab the graph for cars
graph.set_graph('free_flow_time') # let's say we want to minimize time
graph.set_skimming(['free_flow_time', 'distance']) # And will skim time and distance
graph.set_blocked_centroid_flows(False)

D:/src/aequilibrae\aequilibrae\project\network\network.py:239: UserWarning: Fields were removed form Graph for being non-numeric: modes,link_type
  warn(f'Fields were removed form Graph for being non-numeric: {",".join(removed_fields)}')
D:/src/aequilibrae\aequilibrae\paths\graph.py:331: UserWarning: Field name has at least one NaN value.  Your computation may be compromised
  warn(f'Field {i} has at least one NaN value.  Your computation may be compromised')
D:/src/aequilibrae\aequilibrae\paths\graph.py:331: UserWarning: Field lanes has at least one NaN value.  Your computation may be compromised
  warn(f'Field {i} has at least one NaN value.  Your computation may be compromised')


In [42]:
# Let's use the IPF matrix
demand = AequilibraeMatrix()
demand.load(join(fldr, frcst_fldr, 'demand_ipf.omx'))
demand.computational_view() # There is only one matrix there, so don;t even worry about its core name

assig = TrafficAssignment()

# Creates the assignment class
assigclass = TrafficClass(graph, demand)

# The first thing to do is to add at list of traffic classes to be assigned
assig.set_classes([assigclass])

assig.set_vdf("BPR")  # This is not case-sensitive # Then we set the volume delay function

assig.set_vdf_parameters({"alpha": "b", "beta": "power"}) # And its parameters

assig.set_capacity_field("capacity") # The capacity and free flow travel times as they exist in the graph
assig.set_time_field("free_flow_time")

# And the algorithm we want to use to assign
assig.set_algorithm('bfw')

# since I haven't checked the parameters file, let's make sure convergence criteria is good
assig.max_iter = 1000
assig.rgap_target = 0.00001

assig.execute() # we then execute the assignment

2020-03-01 21:51:45,768;aequilibrae;INFO ; bfw Assignment STATS
2020-03-01 21:51:45,769;aequilibrae;INFO ; Iteration, RelativeGap, stepsize
2020-03-01 21:51:48,806;aequilibrae;INFO ; 1,inf,1.0
2020-03-01 21:51:52,046;aequilibrae;INFO ; 2,0.8726912519961871,0.22114875901987208
2020-03-01 21:51:55,580;aequilibrae;INFO ; 3,0.6973287753895054,0.19749958467663412
2020-03-01 21:51:58,853;aequilibrae;INFO ; 4,0.4856088318315293,0.15373984574676489
2020-03-01 21:52:01,819;aequilibrae;INFO ; 5,0.32114802522926184,0.16928534891059838
2020-03-01 21:52:04,771;aequilibrae;INFO ; 6,0.2045914230497074,0.1450133205100151
2020-03-01 21:52:07,854;aequilibrae;INFO ; 7,0.13089541360726356,0.18874364641131797
2020-03-01 21:52:10,914;aequilibrae;INFO ; 8,0.09898479681680672,0.19290433177815644
2020-03-01 21:52:14,299;aequilibrae;INFO ; 9,0.07718934473781647,0.23613187481798378
2020-03-01 21:52:17,572;aequilibrae;INFO ; 10,0.06520310530482806,0.23171318517813286
2020-03-01 21:52:20,649;aequilibrae;INFO ; 11,

2020-03-01 21:56:59,754;aequilibrae;INFO ; 95,0.0006415730886036394,0.08930593540974448
2020-03-01 21:57:03,068;aequilibrae;INFO ; 96,0.0006147640105296731,0.1057390223159807
2020-03-01 21:57:06,577;aequilibrae;INFO ; 97,0.0005630802189359384,0.12991078858560273
2020-03-01 21:57:10,235;aequilibrae;INFO ; 98,0.0005546362644400552,0.10768355877393254
2020-03-01 21:57:13,865;aequilibrae;INFO ; 99,0.0005753763650322795,0.16299364832273736
2020-03-01 21:57:17,189;aequilibrae;INFO ; 100,0.0005532326990176482,0.1464078606513674
2020-03-01 21:57:20,229;aequilibrae;INFO ; 101,0.0005741402147284762,0.2033995906183278
2020-03-01 21:57:23,330;aequilibrae;INFO ; 102,0.0005902441311086241,0.17964706528645244
2020-03-01 21:57:26,313;aequilibrae;INFO ; 103,0.0005519711019764915,0.15714763620433903
2020-03-01 21:57:29,631;aequilibrae;INFO ; 104,0.0005119756564934154,0.15969219644858965
2020-03-01 21:57:33,526;aequilibrae;INFO ; 105,0.0005058108717675543,0.21979911649343117
2020-03-01 21:57:37,335;aequi

2020-03-01 22:02:20,481;aequilibrae;INFO ; 187,0.00016398036945004368,0.021701099416707222
2020-03-01 22:02:23,901;aequilibrae;INFO ; 188,0.0001706248826807003,0.0250943337984337
2020-03-01 22:02:27,419;aequilibrae;INFO ; 189,0.00016555806072750294,0.02106249543120581
2020-03-01 22:02:30,504;aequilibrae;INFO ; 190,0.0001559189206973297,0.01139761320544531
2020-03-01 22:02:33,710;aequilibrae;INFO ; 191,0.00015866237238419353,0.013867074780507487
2020-03-01 22:02:37,132;aequilibrae;INFO ; 192,0.00013036885989456417,0.008922939547392273
2020-03-01 22:02:40,708;aequilibrae;INFO ; 193,0.0001229094168128079,0.008898895865982246
2020-03-01 22:02:44,353;aequilibrae;INFO ; 194,0.00011754203758872814,0.008553373627403835
2020-03-01 22:02:48,139;aequilibrae;INFO ; 195,0.00011900346646248889,0.009609254391761283
2020-03-01 22:02:51,829;aequilibrae;INFO ; 196,0.00011758642469850345,0.009234358345771373
2020-03-01 22:02:55,661;aequilibrae;INFO ; 197,0.00010974646449167704,0.00988041906895231
2020-03

2020-03-01 22:07:31,630;aequilibrae;INFO ; 279,7.588628756229936e-05,0.009937197904953381
2020-03-01 22:07:35,038;aequilibrae;INFO ; 280,6.510775419289805e-05,0.009545609180755968
2020-03-01 22:07:38,308;aequilibrae;INFO ; 281,6.159227639925319e-05,0.0059424629964023624
2020-03-01 22:07:41,618;aequilibrae;INFO ; 282,7.025495263505388e-05,0.007161612674787151
2020-03-01 22:07:44,946;aequilibrae;INFO ; 283,5.939505786361582e-05,0.006682589521116483
2020-03-01 22:07:48,140;aequilibrae;INFO ; 284,5.95308041653753e-05,0.005753982266432482
2020-03-01 22:07:51,301;aequilibrae;INFO ; 285,6.211394480181776e-05,0.006549369837895533
2020-03-01 22:07:54,982;aequilibrae;INFO ; 286,5.9658338297107575e-05,0.005968546048511549
2020-03-01 22:07:58,712;aequilibrae;INFO ; 287,5.5204346979863165e-05,0.006851850429947713
2020-03-01 22:08:02,564;aequilibrae;INFO ; 288,6.425382953842138e-05,0.008538059209452102
2020-03-01 22:08:06,566;aequilibrae;INFO ; 289,6.072418628132919e-05,0.008767009061711137
2020-03-

2020-03-01 22:12:32,491;aequilibrae;INFO ; 370,3.451998893390455e-05,0.004332069921693342
2020-03-01 22:12:36,134;aequilibrae;INFO ; 371,3.6112124281777296e-05,0.004202583596017672
2020-03-01 22:12:39,432;aequilibrae;INFO ; 372,3.722295481428096e-05,0.005048436486803917
2020-03-01 22:12:42,485;aequilibrae;INFO ; 373,3.7119463622728236e-05,0.003949708675230974
2020-03-01 22:12:45,567;aequilibrae;INFO ; 374,3.5196207931818905e-05,0.004784546044547381
2020-03-01 22:12:48,614;aequilibrae;INFO ; 375,3.7143390476780995e-05,0.004721950031870779
2020-03-01 22:12:51,932;aequilibrae;INFO ; 376,3.811059318982423e-05,0.0056234147850315495
2020-03-01 22:12:55,826;aequilibrae;INFO ; 377,4.0747127378504433e-05,0.0048626822932951045
2020-03-01 22:12:59,376;aequilibrae;INFO ; 378,3.644996431095016e-05,0.003435524160164284
2020-03-01 22:13:03,140;aequilibrae;INFO ; 379,3.780210303333328e-05,0.004405401871504968
2020-03-01 22:13:06,592;aequilibrae;INFO ; 380,3.52761294581961e-05,0.003384128475527907
2020

2020-03-01 22:17:37,444;aequilibrae;INFO ; 461,2.44864053935209e-05,0.002437104126191429
2020-03-01 22:17:40,570;aequilibrae;INFO ; 462,2.6062635079979225e-05,0.002529853598113354
2020-03-01 22:17:43,688;aequilibrae;INFO ; 463,2.8157813388550166e-05,0.002989705957703938
2020-03-01 22:17:47,295;aequilibrae;INFO ; 464,2.6101282639649196e-05,0.002801531287863262
2020-03-01 22:17:50,438;aequilibrae;INFO ; 465,2.6379731097477326e-05,0.0027090518581756096
2020-03-01 22:17:53,813;aequilibrae;INFO ; 466,2.8237390191681507e-05,0.002796670478920507
2020-03-01 22:17:57,182;aequilibrae;INFO ; 467,2.6518458336619003e-05,0.0027293017368951516
2020-03-01 22:18:00,257;aequilibrae;INFO ; 468,2.6392493985769388e-05,0.0022531034247533694
2020-03-01 22:18:03,313;aequilibrae;INFO ; 469,2.504597944342173e-05,0.002533149647782807
2020-03-01 22:18:06,481;aequilibrae;INFO ; 470,2.5358031772240732e-05,0.0018657795787258887
2020-03-01 22:18:09,715;aequilibrae;INFO ; 471,2.3644470639314358e-05,0.00284310818939715

2020-03-01 22:23:22,827;aequilibrae;INFO ; 551,1.7477736679413013e-05,0.001694514610208032
2020-03-01 22:23:27,032;aequilibrae;INFO ; 552,1.8907438973865586e-05,0.0020129792493197175
2020-03-01 22:23:30,905;aequilibrae;INFO ; 553,1.898756118512318e-05,0.002307841580932843
2020-03-01 22:23:34,873;aequilibrae;INFO ; 554,2.000933182132417e-05,0.0017407005200264132
2020-03-01 22:23:39,068;aequilibrae;INFO ; 555,1.8360309572113177e-05,0.0019490335872641173
2020-03-01 22:23:43,294;aequilibrae;INFO ; 556,1.8264110237092742e-05,0.0019398490813987033
2020-03-01 22:23:47,267;aequilibrae;INFO ; 557,1.7765872947700908e-05,0.0016319759828332331
2020-03-01 22:23:51,772;aequilibrae;INFO ; 558,1.892853419706048e-05,0.0015119264757312459
2020-03-01 22:23:56,380;aequilibrae;INFO ; 559,1.8548197304503464e-05,0.0020083009338238106
2020-03-01 22:24:01,176;aequilibrae;INFO ; 560,2.015281889416489e-05,0.002512393231856502
2020-03-01 22:24:06,138;aequilibrae;INFO ; 561,2.0428006913024743e-05,0.002203273343260

2020-03-01 22:28:31,451;aequilibrae;INFO ; 641,1.4987748718495298e-05,0.0011938520825935583
2020-03-01 22:28:34,734;aequilibrae;INFO ; 642,1.546623157917248e-05,0.001501308185295552
2020-03-01 22:28:37,886;aequilibrae;INFO ; 643,1.6035250877543015e-05,0.0013249278308930137
2020-03-01 22:28:41,049;aequilibrae;INFO ; 644,1.6978099471215755e-05,0.0016177492175214981
2020-03-01 22:28:44,106;aequilibrae;INFO ; 645,1.683292197588621e-05,0.0015602197789613215
2020-03-01 22:28:47,216;aequilibrae;INFO ; 646,1.6225871706735127e-05,0.001344407244729246
2020-03-01 22:28:50,340;aequilibrae;INFO ; 647,1.715339991232932e-05,0.001265146118620634
2020-03-01 22:28:53,683;aequilibrae;INFO ; 648,1.440512313411523e-05,0.0011832335305578662
2020-03-01 22:28:57,211;aequilibrae;INFO ; 649,1.4599583066427417e-05,0.0010680290428995343
2020-03-01 22:29:00,812;aequilibrae;INFO ; 650,1.5942083290686554e-05,0.000906182168729135
2020-03-01 22:29:04,126;aequilibrae;INFO ; 651,1.7540830872464836e-05,0.0012017273359284

2020-03-01 22:33:32,282;aequilibrae;INFO ; 731,1.4094864866537674e-05,0.0015296077796327749
2020-03-01 22:33:35,794;aequilibrae;INFO ; 732,1.646654735116761e-05,0.0019167856972220848
2020-03-01 22:33:39,229;aequilibrae;INFO ; 733,1.4511530786949079e-05,0.0014089791140847333
2020-03-01 22:33:42,617;aequilibrae;INFO ; 734,1.575940997759709e-05,0.001749032890181565
2020-03-01 22:33:45,920;aequilibrae;INFO ; 735,1.4338122639979527e-05,0.001651932885680511
2020-03-01 22:33:49,542;aequilibrae;INFO ; 736,1.6514387142167504e-05,0.0017339562498487724
2020-03-01 22:33:53,296;aequilibrae;INFO ; 737,1.4713973671327853e-05,0.001691422299114825
2020-03-01 22:33:57,033;aequilibrae;INFO ; 738,1.5144885380304864e-05,0.0022018647114493144
2020-03-01 22:34:00,671;aequilibrae;INFO ; 739,1.5303828184360615e-05,0.002262588001496278
2020-03-01 22:34:04,478;aequilibrae;INFO ; 740,1.684397185907419e-05,0.002556690795098819
2020-03-01 22:34:07,980;aequilibrae;INFO ; 741,1.526976651918316e-05,0.00245303652044362

2020-03-01 22:38:47,079;aequilibrae;INFO ; 821,1.2937098323725375e-05,0.0013326749094983814
2020-03-01 22:38:50,467;aequilibrae;INFO ; 822,1.3111166042813497e-05,0.0009624243859350319
2020-03-01 22:38:53,961;aequilibrae;INFO ; 823,1.2228985942284457e-05,0.0013710768827122973
2020-03-01 22:38:57,514;aequilibrae;INFO ; 824,1.341421052380198e-05,0.0010570031742713874
2020-03-01 22:39:00,834;aequilibrae;INFO ; 825,1.2997405698991423e-05,0.0010718472221833576
2020-03-01 22:39:04,160;aequilibrae;INFO ; 826,1.2232777301825034e-05,0.0011473962721528779
2020-03-01 22:39:07,463;aequilibrae;INFO ; 827,1.2464992421737865e-05,0.0010988835023077418
2020-03-01 22:39:10,778;aequilibrae;INFO ; 828,1.2929546860347815e-05,0.0012575323707039584
2020-03-01 22:39:14,085;aequilibrae;INFO ; 829,1.234426737822347e-05,0.001186034578338723
2020-03-01 22:39:17,341;aequilibrae;INFO ; 830,1.2462075501603352e-05,0.0010214396695460363
2020-03-01 22:39:20,647;aequilibrae;INFO ; 831,1.2667134555748677e-05,0.00112634827

In [43]:
# The link flows are easy to export.
# we do so for csv and AequilibraEData
assigclass.results.save_to_disk(join(fldr, ftr_fldr, 'future_link_flows_c.csv'), output="loads")
assigclass.results.save_to_disk(join(fldr, ftr_fldr, 'future_link_flows_c.aed'), output="loads")

# the skims are easy to get.

# The blended one are here
avg_skims = assigclass.results.skims

# The ones for the last iteration are here
last_skims = assigclass._aon_results.skims

# Assembling a single final skim file can be done like this
# We will want only the time for the last iteration and the distance averaged out for all iterations
kwargs = {'file_name': join(fldr,ftr_fldr, 'future_skims.aem'),
          'zones': graph.num_zones,
          'matrix_names': ['time_final', 'distance_blended']}

# Create the matrix file
out_skims = AequilibraeMatrix()
out_skims.create_empty(**kwargs)
out_skims.index[:] = avg_skims.index[:]

# Transfer the data
 # The names of the skims are the name of the fields
out_skims.matrix['time_final'][:,:] = last_skims.matrix['free_flow_time'][:,:]
# It is CRITICAL to assign the matrix values using the [:,:]
out_skims.matrix['distance_blended'][:,:] = avg_skims.matrix['distance'][:,:]

out_skims.matrices.flush() # Make sure that all data went to the disk

# Export to OMX as well
out_skims.export(join(fldr,ftr_fldr, 'future_skims.omx'))
